In [10]:
from datasets import load_dataset

def build_filtered_test_dataset(dataset_name='derek-thomas/ScienceQA',
                                 split='test',
                                 keep_grades='1-6'):
    """
    构建按年级过滤的测试数据集（不限制样本数量）。

    参数:
        dataset_name (str): 数据集名称，例如 'derek-thomas/ScienceQA'。
        split (str): 数据分割，例如 'train', 'test', 'validation'。
        keep_grades (str or None): 筛选的年级段："1-6"、"7-12" 或 None 表示不过滤。

    返回:
        List[Dict]: 筛选后的样本列表。
    """

    def is_grade_allowed(grade_str):
        if keep_grades is None:
            return True
        try:
            grade_num = int(grade_str.replace("grade", ""))
            if keep_grades == "1-6":
                return 1 <= grade_num <= 6
            elif keep_grades == "7-12":
                return 7 <= grade_num <= 12
        except:
            return False
        return False

    data = load_dataset(dataset_name, split=split)
    test_dataset = []

    for i, sample in enumerate(data):
        try:
            if sample.get('question') is None:
                continue

            if not is_grade_allowed(sample.get("grade", "")):
                continue

            solution = sample.get("solution", "")
            lecture = sample.get("lecture", "")
            solution_lecture = f"{solution}\n\n{lecture}".strip()

            test_dataset.append({
                "image": sample.get("image", None), 
                "question": sample["question"],
                "choices": sample["choices"],
                "hint": sample["hint"],
                "answer": sample["answer"],
                "solution_lecture": solution_lecture,
                'grade':sample["grade"],
            })

        except Exception as e:
            print(f"跳过第 {i} 个样本，错误：{e}")
            continue

    return test_dataset




In [11]:
# 只保留 grade1 到 grade6 的前 10 个样本
data = build_filtered_test_dataset(dataset_name='derek-thomas/ScienceQA',
                                    split='validation',
                                    keep_grades='7-12')

# 只保留 grade7 到 grade12
# filtered_data = build_filtered_test_dataset(N=10, keep_grades='7-12')

# # 不进行年级过滤
# filtered_data = build_filtered_test_dataset(N=10, keep_grades=None)


In [13]:
data

[{'image': None,
  'question': "What does the verbal irony in this text suggest?\nAccording to Mr. Herrera's kids, his snoring is as quiet as a jackhammer.",
  'choices': ['The snoring is loud.', 'The snoring occurs in bursts.'],
  'hint': '',
  'answer': 0,
  'solution_lecture': "The text uses verbal irony, which involves saying one thing but implying something very different.\nAs quiet as a jackhammer suggests that the snoring is loud. A jackhammer is not quiet, and neither is Mr. Herrera's snoring.\n\nFigures of speech are words or phrases that use language in a nonliteral or unusual way. They can make writing more expressive.\nVerbal irony involves saying one thing but implying something very different. People often use verbal irony when they are being sarcastic.\nOlivia seems thrilled that her car keeps breaking down.\nEach breakdown is as enjoyable as a punch to the face.",
  'grade': 'grade8'},
 {'image': None,
  'question': 'Is this a sentence fragment?\nDuring the construction

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def initialize_data_loaders(batch_size=64, download=False):
    """初始化并返回训练集和测试集的数据加载器"""
    # 定义数据转换操作
    transform = transforms.ToTensor()

    # 准备MNIST数据集
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=download)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=download)

    # 数据加载器
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

class SimpleNN(nn.Module):
    """定义简单的神经网络结构"""
    def __init__(self, input_size=784, num_classes=10):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, num_classes)
  
    def forward(self, x):
        x = x.view(x.size(0), -1)  # 展平图像为一维向量
        return torch.softmax(self.fc1(x), dim=1)

def train(model, loader, optimizer, loss_fn):
    """训练模型"""
    model.train()
    total_loss = 0
    for inputs, labels in loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'训练损失: {total_loss / len(loader):.4f}')

def test(model, loader):
    """测试模型"""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'测试准确率: {100 * correct / total:.2f}%')